In [ ]:
import os 
import sys
sys.path.append(os.path.join(os.getcwd(), '..'))
import pytorch_lightning as pl
from model.model_interface import LLM
from dataset.addition import Addition
import torch.utils.data as tud
from lightning.pytorch.loggers import TensorBoardLogger
import os 
import random


生成训练测试集

In [ ]:
ub = 3
data_size = 200000
test_size = 0.6
lines = set()

# for i in range(10**ub):
#     for j in range(10**ub-i):
#         i = str(i).zfill(ub)
#         lines.add(f'{i}+{j}={i+j})
while len(lines) < data_size:
    l1 = random.randint(1, ub)
    l2 = random.randint(1, ub)
    a = random.randint(10**(l1-1), 10**l1-1)
    b_ub = 10**(ub) - a - 1
    b = random.randint(min(10**(l2-1), b_ub), min(10**l2, b_ub))
    c = a+b
    a = str(a).zfill(ub)
    b = str(b).zfill(ub)
    c = str(c).zfill(ub)
    lines.add(f'{a}+{b}={c}\n')

lines = list(lines)
random.shuffle(lines)
with open(f'data/train_{ub}_zfill.txt','w') as f:
    f.writelines(lines[int(len(lines)*test_size):])
with open(f'data/test_{ub}_zfill.txt','w') as f:
    f.writelines(lines[:int(len(lines)*test_size)])

训练配置

In [ ]:
# LLM Config
llm_config = {
    "model_name": "gpt2math_small",
    "optimizer": "adam_w",
    "weight_decay": 1.0,
    "lr": 1e-4,
}

# Dataset config
dl_config = {
    "batch_size": 1024,
    "num_workers": 0,
}
data_dir= "data"
train_file= "train_2_zfill.txt"
test_file= "test_2_zfill.txt"

# Trainer config
trainer_config = {
    "precision" : "16-mixed",
    "accelerator" : "auto",
    "devices" : 1,
    "max_epochs": 500,
}

初始化模型，加载数据集，训练器

In [ ]:
mt = LLM(**llm_config)
train_dl = tud.DataLoader(Addition(f"{data_dir}/{train_file}", mt.tokenizer), shuffle=True, **dl_config )
test_dl = tud.DataLoader(Addition(f"{data_dir}/{test_file}", mt.tokenizer), shuffle=False, **dl_config)
# default logger used by trainer (if tensorboard is installed)
logger = TensorBoardLogger(save_dir=os.getcwd(), version=1, name="lightning_logs")
trainer = pl.Trainer(**trainer_config, logger=logger)

开始训练！

In [ ]:
trainer.fit(mt,train_dl,test_dl,)

测试

In [ ]:
test_res = trainer.test(mt,test_dl,)